# Numbers (and everything else) are represented in a computer in binary.
There are two main types of numbers **integer** numbers and **floating-point** numbers.

**Integer** numbers are represented exactly by their base 2 bit sequence. Using 32-bits signed **integer**
values in the range -2147483648 to 2147483647 can be represented.

**Floating-point** numbers use a special format that approximates a real number.

In [43]:
# Include package useful for examining float number bit patterns
import struct

In [3]:
# Lets set an integer value and look at its bit pattern
# Python includes a standard function bin() that returns the binary representation of an integer.
i=4
bin(i)

'0b100'

In [4]:
# We can read about the bin() function using the ? prefix
?bin

Signature: bin(number, /)
Docstring:
Return the binary representation of an integer.

>>> bin(2796202)
'0b1010101010101010101010'
Type:      builtin_function_or_method


### A real world example ( https://en.wikipedia.org/wiki/Electronic_voting_in_Belgium#cite_note-2 ) of bits!
In a Belgium election with early electronic voting machines one candidate was mysteriously tallied with an extra 4096 votes.
This was inconsistent with the number of people voting. After investigating it was assumed that the machine, which
did not have error correcting memory, had been affected by a cosmic ray particle flipping "bit 12" of a 32-bit number! 

In [40]:
# Set a value
i=20
s="{:032b}".format(i)
print("    s =",s)

# "Flip" a bit
iflip=i | 0b00000000000000000001000000010100   # use the vertical bar "|" which means logical "or" the two values

# Examine the results
sflip="{:032b}".format(iflip)
print("sflip =",sflip)
print("iflip =",iflip)
print("iflip - i =",iflip - i)

    s = 00000000000000000000000000010100
sflip = 00000000000000000001000000010100
iflip = 4116
iflip - i = 4096


### Floating point numbers are more involved
A floating point number is a value that can have some fractional part e.g. 1.3 .
Floating point numbers are represented in a format (describe in class ). In Python floating point numbers
use a 64-bit representation by default ( see class notes and https://en.wikipedia.org/wiki/Double-precision_floating-point_format ). 

We can examine this representation using the Python "struct" package. Fully understanding the "struct" package is a little
involved, here we use it as a means to generate the underlying computer hardware representation of a floating point number without
going into all its details.

In [90]:
# Set a floating point number ( in many languages we can add a decimal point to indicate a number
# is to be treated as floating point - even if it is a whole number.  This makes then computer store the
# number using the floating point format). 
x=12.

# Use "struct" to generate the underlying binary representation
# and print the result
s="".join([f"{b:08b}" for b in struct.pack(">d", x)])
print(s)

# Now lets look at the parts
s_sign=s[0:0+1]
s_biased_exponent=s[1:1+11]
s_mantissa=s[12:12+52]
print( "sign =", s_sign, "\nbiased exponent =", s_biased_exponent, "\nmantissa = ",  s_mantissa )

0100000000101000000000000000000000000000000000000000000000000000
sign = 0 
biased exponent = 10000000010 
mantissa =  1000000000000000000000000000000000000000000000000000


### Lets check how it works!

In [91]:
# What is the exponent part
epart=2**(0b10000000010-1023)
print(epart)

8


In [92]:
# What is the mantissa part
v=0.5
mpart=1.
for c in s_mantissa:
    if c == '1':
     mpart=mpart+v
    v=v*0.5
print(mpart)

1.5


In [93]:
# What is exponent x mantissa
print(mpart*epart)

12.0


In [87]:
# 1. what happens if we flip the sign bit
s_new=0b1100000000101000000000000000000000000000000000000000000000000000
struct.unpack('>d',int.to_bytes(s_new,8,'big'))

(-12.0,)

In [88]:
# 2. what happens if we flip the last bit of the mantissa
s_new=0b0100000000101000000000000000000000000000000000000000000000000001
struct.unpack('>d',int.to_bytes(s_new,8,'big'))

(12.000000000000002,)

In [89]:
# 3. what happens if we flip the least significant bit of the exponent
s_new=0b0100000000111000000000000000000000000000000000000000000000000000
struct.unpack('>d',int.to_bytes(s_new,8,'big'))

(24.0,)

In [97]:
# 4. what happens if we flip the most significant bit of the exponent
s_new=0b0000000000101000000000000000000000000000000000000000000000000000
struct.unpack('>d',int.to_bytes(s_new,8,'big'))

(6.675221575521604e-308,)

### Fortunately cosmic rays are rare and most computer hardware includes some sort of error correcting bits in memory. So bit flips very rarely happen - we think! 
### On systems like the Mars helicopter radiation protection and multiple copies of same hardware are sometimes used to guard against bit-flips. Space hardware also typically uses older, larger transistors that have more elctrons in each memory cell. 

In [ ]:
#